In [15]:
%pip install Pillow
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import nn, optim
import torch

# Transforms
transforms = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize grayscale images
])

# Load Datasets
train_dataset = datasets.ImageFolder(root=r'c:\Users\Dell\OneDrive\Desktop\database\Emotion_detection_dataset\archive (5)\train', transform=transforms)
test_dataset = datasets.ImageFolder(root=r'c:\Users\Dell\OneDrive\Desktop\database\Emotion_detection_dataset\archive (5)\test', transform=transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

# Define CNN Model
class CNN_emotion(nn.Module):
    def __init__(self):
        super(CNN_emotion, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)  # 46x46
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 23x23
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)  # 21x21
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)  # 19x19
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)  # 17x17
        self.fc1 = nn.Linear(64 * 17 * 17, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 250)
        self.fc4 = nn.Linear(250, 100)
        self.fc5 = nn.Linear(100, 50)
        self.fc6 = nn.Linear(50, 7)  # 7 emotions in the dataset
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Add dropout to prevent overfitting

    def forward(self, X):
        X = self.conv1(X)
        X = self.relu(self.maxpool1(X))
        X = self.relu(self.conv2(X))
        X = self.relu(self.conv3(X))
        X = self.relu(self.conv4(X))
        X = X.view(X.size(0), -1)  # Flatten
        X = self.relu(self.fc1(X))
        X = self.dropout(self.relu(self.fc2(X)))  # Apply dropout
        X = self.dropout(self.relu(self.fc3(X)))
        X = self.relu(self.fc4(X))
        X = self.relu(self.fc5(X))
        X = self.fc6(X)  # Output logits (no softmax)
        return X

# Model Initialization
model = CNN_emotion()
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss includes softmax
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epoch = 100
for epoch in range(1, num_epoch + 1):
    model.train()  # Set the model to training mode
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"{epoch}/{num_epoch}: Loss: {total_loss:.4f}")

# Evaluation Function
def evaluate(loader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='micro')
    recall = recall_score(all_labels, all_preds, average='micro')
    f1 = f1_score(all_labels, all_preds, average='micro')
    
    return accuracy, precision, recall, f1

# Training Metrics
train_acc, train_prec, train_recall, train_f1 = evaluate(train_loader)
print("***Training***")
print(f"Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_recall:.4f}, F1-Score: {train_f1:.4f}")

# Testing Metrics
test_acc, test_prec, test_recall, test_f1 = evaluate(test_loader)
print("***Testing***")
print(f"Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_recall:.4f}, F1-Score: {test_f1:.4f}")


1/100: Loss: 757.9316
2/100: Loss: 670.2781
3/100: Loss: 606.1232
4/100: Loss: 536.5504
5/100: Loss: 447.7969
6/100: Loss: 333.7928
7/100: Loss: 235.7918
8/100: Loss: 158.8545
9/100: Loss: 109.1971
10/100: Loss: 82.1176
11/100: Loss: 62.9519
12/100: Loss: 50.3633
13/100: Loss: 39.5514
14/100: Loss: 40.8752
15/100: Loss: 34.9670
16/100: Loss: 31.6960
17/100: Loss: 27.1904
18/100: Loss: 26.1686
19/100: Loss: 27.1647
20/100: Loss: 24.1140
21/100: Loss: 24.3064
22/100: Loss: 23.9868
23/100: Loss: 22.5779
24/100: Loss: 23.7655
25/100: Loss: 20.1892
26/100: Loss: 20.5970
27/100: Loss: 20.3171
28/100: Loss: 20.2643
29/100: Loss: 16.3491
30/100: Loss: 16.8938
31/100: Loss: 17.8534
32/100: Loss: 17.1285
33/100: Loss: 14.7586
34/100: Loss: 17.3986
35/100: Loss: 16.3935
36/100: Loss: 11.6847
37/100: Loss: 20.5861
38/100: Loss: 14.4476
39/100: Loss: 15.9337
40/100: Loss: 16.3170
41/100: Loss: 12.2792
42/100: Loss: 15.7578
43/100: Loss: 12.3131
44/100: Loss: 12.1108
45/100: Loss: 11.6769
46/100: Lo

In [2]:
# Save the model after training
torch.save(model.state_dict(), 'emotion_detection_model.pth')
print("Model saved as 'emotion_detection_model.pth'")

Model saved as 'emotion_detection_model.pth'
